## Feature Engineering: Housing data project
#### I don't really know what feature engineering is but this is where I am implementing my strategy and creating new collumns that are mathematically derived from 1 or more collumns describing an attribute and some measure of that attribute

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression, RidgeCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer,make_column_transformer
from a02_as_funct import map_home, numeric_houses, dummy_houses, multiplier_attempt1, multiplier_attempt2, multiplier_attempt3

### read raw data

In [2]:
houses = pd.read_csv('../datasets/train.csv')
houses.head()
houses.columns =houses.columns.str.lower().str.replace(' ','_')

y=houses['saleprice']

### Clean, map the attribute and modifier collumns when needed, ensure datatypes, ordinally encode
#### I built a series of functions that rapidly do this for data shaped like the data we are provided 

In [3]:
houses1 = map_home(houses.drop(columns=['saleprice']))

houses2 = numeric_houses(houses1) # not dummified yet
X = houses2
#houses3 = dummy_houses(houses2)

In [4]:
houses2.columns

Index(['pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area', 'street',
       'alley', 'land_contour', 'utilities', 'lot_config', 'land_slope',
       'neighborhood', 'condition_1', 'condition_2', 'bldg_type',
       'house_style', 'overall_qual', 'overall_cond', 'year_built',
       'year_remod/add', 'roof_style', 'roof_matl', 'exterior_1st',
       'exterior_2nd', 'mas_vnr_type', 'mas_vnr_area', 'exter_qual',
       'exter_cond', 'foundation', 'bsmt_qual', 'bsmt_cond', 'bsmt_exposure',
       'bsmtfin_type_1', 'bsmtfin_sf_1', 'bsmtfin_type_2', 'bsmtfin_sf_2',
       'bsmt_unf_sf', 'total_bsmt_sf', 'heating', 'heating_qc', 'central_air',
       'electrical', '1st_flr_sf', '2nd_flr_sf', 'low_qual_fin_sf',
       'gr_liv_area', 'bsmt_full_bath', 'bsmt_half_bath', 'full_bath',
       'half_bath', 'bedroom_abvgr', 'kitchen_abvgr', 'kitchen_qual',
       'totrms_abvgrd', 'functional', 'fireplaces', 'fireplace_qu',
       'garage_type', 'garage_yr_blt', 'garage_finish', 'garage_car

In [5]:
houses2.head()

,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,land_contour,utilities,lot_config,...,screen_porch,pool_area,pool_qc,misc_feature,misc_val,mo_sold,yr_sold,sale_type,fence_private,fence_wood
0,533352170,60,RL,0.0,13517,Pave,NaN,Lvl,AllPub,CulDSac,...,0,0,0,NaN,0,3,2010,WD,0,0
1,531379050,60,RL,43.0,11492,Pave,NaN,Lvl,AllPub,CulDSac,...,0,0,0,NaN,0,4,2009,WD,0,0
2,535304180,20,RL,68.0,7922,Pave,NaN,Lvl,AllPub,Inside,...,0,0,0,NaN,0,1,2010,WD,0,0
3,916386060,60,RL,73.0,9802,Pave,NaN,Lvl,AllPub,Inside,...,0,0,0,NaN,0,4,2010,WD,0,0
4,906425045,50,RL,82.0,14235,Pave,NaN,Lvl,AllPub,Inside,...,0,0,0,NaN,0,3,2010,WD,0,0


In [6]:
#ct.get_feature_names_out()
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=512)


In [7]:
# Similiar to a technique shared by Katie Ellsweig over Zoom chat in GA-DSI911
cat_features = ['mas_vnr_type','bsmt_exposure','garage_type','alley','misc_feature',
                'land_contour','ms_subclass','ms_zoning','street','neighborhood',
                'condition_2','bldg_type', 'house_style','roof_style','roof_matl',
                'exterior_1st','exterior_2nd','foundation',
               'central_air','electrical','sale_type','condition_1','utilities','lot_config','land_slope',
               'heating','paved_drive','mo_sold']
ct = ColumnTransformer([('oh', OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first'), cat_features)
], remainder='passthrough')

In [8]:
X_train_dummy = pd.DataFrame(ct.fit_transform(X_train),columns=ct.get_feature_names_out())
X_test_dummy = pd.DataFrame(ct.transform(X_test),columns=ct.get_feature_names_out())

C:\Users\seanh\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [4, 7, 14, 15, 19, 22] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [9]:
#ct.get_feature_names_out()

In [10]:
X_train_dummy.shape

(1538, 224)

In [11]:
X_test_dummy.shape

(513, 224)

In [12]:
ct.get_feature_names_out

<bound method ColumnTransformer.get_feature_names_out of ColumnTransformer(remainder='passthrough',
                  transformers=[('oh',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['mas_vnr_type', 'bsmt_exposure',
                                  'garage_type', 'alley', 'misc_feature',
                                  'land_contour', 'ms_subclass', 'ms_zoning',
                                  'street', 'neighborhood', 'condition_2',
                                  'bldg_type', 'house_style', 'roof_style',
                                  'roof_matl', 'exterior_1st', 'exterior_2nd',
                                  'foundation', 'central_air', 'electrical',
                                  'sale_type', 'condition_1', 'utilities',
                                  'lot_config', 'land_slop

###### 

### Now I need to mathematically combine the collumns I mapped out at the start of this 

In [13]:
X_train_multiplied = multiplier_attempt3(X_train_dummy)
X_test_multiplied = multiplier_attempt3(X_test_dummy)

+,

In [14]:
## standard scale
sc = StandardScaler()
X_train_multiplied_sc=sc.fit_transform(X_train_multiplied)
X_test_multiplied_sc=sc.transform(X_test_multiplied)

## Building a linear model. does very well on the test data with an r2 score of 95%. Does poorly on test data - 54% the model is overfit

## KnRegression n_neighbors optimization using GridSearchCV
#### heavily influenced by lesson 406 at GA-DSI

In [15]:
pgrid= {'n_neighbors':np.arange(2,100)}

In [16]:
knn = KNeighborsRegressor()

In [17]:
gs= GridSearchCV(estimator=knn,param_grid=pgrid)


In [18]:
%%time
gs.fit(X_train_multiplied_sc,y_train)

CPU times: total: 45.3 s
Wall time: 14.1 s


GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
       87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [19]:
gs.score(X_train_multiplied_sc,y_train)

0.8087121782274681

In [20]:
gs.score(X_test_multiplied_sc,y_test)

0.7998126314608629

In [21]:
X_test_multiplied_sc.shape

(513, 234)

In [22]:
gs.best_estimator_

KNeighborsRegressor(n_neighbors=11)

## Before changes to the mathematical operations i have found 12 is the best k value now processes the entire training dataset, afterwards 4 seems optimal

In [23]:
# redoing this as before for all of X
"""cat_features = ['mas_vnr_type','bsmt_exposure','garage_type','alley','misc_feature',
                'land_contour','ms_subclass','ms_zoning','street','neighborhood',
                'condition_2','bldg_type', 'house_style','roof_style','roof_matl',
                'exterior_1st','exterior_2nd','foundation',
               'central_air','electrical','sale_type','condition_1','utilities','lot_config','land_slope',
               'heating','paved_drive']"""
#ct = ColumnTransformer([('oh', OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first'), cat_features)
#], remainder='passthrough')
X_dummy =pd.DataFrame(ct.fit_transform(X),columns=ct.get_feature_names_out())
X_multiplied = multiplier_attempt3(X_dummy)
X_multiplied_sc=sc.fit_transform(X_multiplied)
knn = KNeighborsRegressor(n_neighbors=4)
knn.fit(X_multiplied_sc,y)

KNeighborsRegressor(n_neighbors=4)

## time to deal with confounding variables: parsimony please

In [24]:
numeric_cols= list(X.corr(numeric_only=True).columns)

In [25]:
pd.DataFrame(data=np.where(np.abs(X.corr(numeric_only=True))>.85,1,0),index = numeric_cols,columns=numeric_cols ).sum()

pid                1
ms_subclass        1
lot_frontage       1
lot_area           1
overall_qual       1
overall_cond       1
year_built         1
year_remod/add     1
mas_vnr_area       1
exter_qual         1
exter_cond         1
bsmt_qual          1
bsmt_cond          1
bsmtfin_type_1     1
bsmtfin_sf_1       1
bsmtfin_type_2     1
bsmtfin_sf_2       1
bsmt_unf_sf        1
total_bsmt_sf      1
heating_qc         1
1st_flr_sf         1
2nd_flr_sf         1
low_qual_fin_sf    1
gr_liv_area        1
bsmt_full_bath     1
bsmt_half_bath     1
full_bath          1
half_bath          1
bedroom_abvgr      1
kitchen_abvgr      1
kitchen_qual       1
totrms_abvgrd      1
functional         1
fireplaces         2
fireplace_qu       2
garage_yr_blt      3
garage_finish      1
garage_cars        2
garage_area        2
garage_qual        3
garage_cond        3
wood_deck_sf       1
open_porch_sf      1
enclosed_porch     1
3ssn_porch         1
screen_porch       1
pool_area          2
pool_qc      

## A few of these have high correlations (over 90%) with other X variables
#### maybe when I calculate my new columns I can deal with a few of these and adjust
```
pool_area          2
pool_qc            2
garage_yr_blt      3
```
#### I adjusted my ordinal encoding formulas which minimally lowered my kaggle score. I think theres too much noise so I also now remove columns highly correlated or that do nothing but modify the value of other features

## A few of these have high correlations (over 85%) with other X variables
pool_area          2
pool_qc            2
garage_cars        2
garage_area        2
garage_qual        3
garage_cond        3
fireplaces         2
fireplace_qu       2
garage_yr_blt      3

In [26]:
X_multiplied_sc.shape

(2051, 241)

In [27]:
knn.score(X_multiplied_sc,y)# knregressor is trained and process is repeated for all data (hopefully) without leakage

0.8645231403442396

In [28]:
#knn.score(X_multiplied_sc,y)

In [29]:
test = pd.read_csv('../datasets/test.csv')
test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [30]:
test1 = map_home(test)

X_kaggle = numeric_houses(test1) # not dummified yet

#ct.get_feature_names_out()
#X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=512)
# Similiar to a technique shared by Katie Ellsweig over Zoom chat in GA-DSI911
"""cat_features = ['mas_vnr_type','bsmt_exposure','garage_type','alley','misc_feature',
                'land_contour','ms_subclass','ms_zoning','street','neighborhood',
                'condition_2','bldg_type', 'house_style','roof_style','roof_matl',
                'exterior_1st','exterior_2nd','foundation',
               'central_air','electrical','sale_type','condition_1','utilities','lot_config','land_slope',
               'heating','paved_drive']
ct = ColumnTransformer([('oh', OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first'), cat_features)
], remainder='passthrough')"""
X_kaggle_dummy = pd.DataFrame(ct.transform(X_kaggle),columns=ct.get_feature_names_out())
#X_test_dummy = pd.DataFrame(ct.transform(X_test),columns=ct.get_feature_names_out())
X_kaggle_multiplied = multiplier_attempt3(X_kaggle_dummy)
#X_test_multiplied = multiplier_attempt3(X_test_dummy)
## standard scale
#sc = StandardScaler()
X_kaggle_multiplied_sc=sc.transform(X_kaggle_multiplied)
#X_test_multiplied_sc=sc.transform(X_test_multiplied)

C:\Users\seanh\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0, 14, 15, 16, 19, 20, 25] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [31]:
X_kaggle_multiplied_sc.shape

(878, 241)

In [32]:
y_pred = knn.predict(X_kaggle_multiplied_sc)

In [33]:
data={'Id': test.id , 'SalePrice': y_pred}

df = pd.DataFrame(data)

df.to_csv('../datasets/kaggle_sub_5.csv', index=False)

#df = pd.read_csv('../datasets/kaggle_sub_3.csv')
df.head()

,Id,SalePrice
0,2658,119950.00
1,2718,133625.00
2,2414,180416.25
3,1989,83850.00
4,625,148625.00


In [34]:
r_alphas = np.logspace(-2,3,100)
ridgecv = RidgeCV(alphas=r_alphas, cv=10)

In [35]:
X_train_multiplied.shape

(1538, 234)

In [36]:
X_multiplied_sc.shape

(2051, 241)

In [37]:
X_kaggle_multiplied_sc.shape

(878, 241)

In [38]:
%%time
ridgecv.fit(X_train_multiplied_sc,y_train)

CPU times: total: 17.9 s
Wall time: 4.99 s


RidgeCV(alphas=array([1.00000000e-02, 1.12332403e-02, 1.26185688e-02, 1.41747416e-02,
       1.59228279e-02, 1.78864953e-02, 2.00923300e-02, 2.25701972e-02,
       2.53536449e-02, 2.84803587e-02, 3.19926714e-02, 3.59381366e-02,
       4.03701726e-02, 4.53487851e-02, 5.09413801e-02, 5.72236766e-02,
       6.42807312e-02, 7.22080902e-02, 8.11130831e-02, 9.11162756e-02,
       1.02353102e-01, 1.14975700e-0...
       6.89261210e+01, 7.74263683e+01, 8.69749003e+01, 9.77009957e+01,
       1.09749877e+02, 1.23284674e+02, 1.38488637e+02, 1.55567614e+02,
       1.74752840e+02, 1.96304065e+02, 2.20513074e+02, 2.47707636e+02,
       2.78255940e+02, 3.12571585e+02, 3.51119173e+02, 3.94420606e+02,
       4.43062146e+02, 4.97702356e+02, 5.59081018e+02, 6.28029144e+02,
       7.05480231e+02, 7.92482898e+02, 8.90215085e+02, 1.00000000e+03]),
        cv=10)

In [39]:
ridgecv.score(X_test_multiplied_sc,y_test)

0.8511889102267705

## after this step I started over in notebook 04 to begin the real work

In [40]:
#y_pred = ridgecv.predict(X_kaggle_multiplied_sc)

In [41]:
#data={'Id': test.id , 'SalePrice': y_pred}

#df = pd.DataFrame(data)

#df.to_csv('../datasets/kaggle_sub_6_ridge.csv', index=False)

#df = pd.read_csv('../datasets/kaggle_sub_3.csv')
#df.head()

### Testing to be streaming as a function

### Kaggle result submitting too